<a href="https://colab.research.google.com/github/epvillanueva10/Datos/blob/main/P1_ImagePreprocessing_Py6S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Atmospheric correction of Sentinel 2 image using Py6S in Google Colab environment**


Guidance and reference provided at the following websites are appreciated.

*   https://github.com/samsammurphy/gee-atmcorr-S2
*   https://github.com/ndminhhus/geeguide/blob/master/02.Atm-correction.md
*   https://blog.csdn.net/qq_45110581/article/details/108629636
*   https://github.com/ivanhykwong/marine-water-quality-time-series-hk








# Step 1 - Set up Py6S in Google Colab

In [ ]:
!gfortran -v
!wget http://rtwilson.com/downloads/6SV-1.1.tar
!tar xvf 6SV-1.1.tar
!cd 6SV1.1

Using built-in specs.
COLLECT_GCC=gfortran
COLLECT_LTO_WRAPPER=/usr/lib/gcc/x86_64-linux-gnu/11/lto-wrapper
OFFLOAD_TARGET_NAMES=nvptx-none:amdgcn-amdhsa
OFFLOAD_TARGET_DEFAULT=1
Target: x86_64-linux-gnu
Configured with: ../src/configure -v --with-pkgversion='Ubuntu 11.4.0-1ubuntu1~22.04' --with-bugurl=file:///usr/share/doc/gcc-11/README.Bugs --enable-languages=c,ada,c++,go,brig,d,fortran,objc,obj-c++,m2 --prefix=/usr --with-gcc-major-version-only --program-suffix=-11 --program-prefix=x86_64-linux-gnu- --enable-shared --enable-linker-build-id --libexecdir=/usr/lib --without-included-gettext --enable-threads=posix --libdir=/usr/lib --enable-nls --enable-bootstrap --enable-clocale=gnu --enable-libstdcxx-debug --enable-libstdcxx-time=yes --with-default-libstdcxx-abi=new --enable-gnu-unique-object --disable-vtable-verify --enable-plugin --enable-default-pie --with-system-zlib --enable-libphobos-checking=release --with-target-system-zlib=auto --enable-objc-gc=auto --enable-multiarch --disab

**Manual work required before executing the subsequent code**

Refer to comments below

In [ ]:
# modify "makefile" from "FC = g77 $(FFLAGS)" to "FC = gfortran -std=legacy -ffixed-line-length-none -ffpe-summary=none $(FFLAGS)"
# upload modified "makefile" to /content/6SV1.1

import os
os.chdir("/content/6SV1.1")
!ls
!make
os.environ["PATH"]="/content/6SV1.1:"+os.environ["PATH"]
# test
!sixsV1.1 < /content/Examples/Example_In_1.txt
!pip install Py6S
from Py6S import *
SixS.test()

6SV1.1	       CSALBR.f     INTERP.f	 MOCA1.f	 ODA550.f      PRESPLANE.f    TM.f
6SV-1.1.tar    DICA1.f	    ISO.f	 MOCA2.f	 ODRAYL.f      PRESSURE.f     TROPIC.f
6SV-1.1.tar.1  DICA2.f	    KERNEL.f	 MOCA3.f	 OS.f	       PRINT_ERROR.f  TRUNCA.f
6SV-1.1.tar.2  DICA3.f	    KERNELPOL.f  MOCA4.f	 OSPOL.f       RAHMALBE.f     US62.f
6SV-1.1.tar.3  DISCOM.f     LAKEW.f	 MOCA5.f	 OXYG3.f       RAHMBRDF.f     VARSOL.f
6SV-1.1.tar.4  DISCRE.f     main.f	 MOCA6.f	 OXYG4.f       ROUJALBE.f     VEGETA.f
6SV-1.1.tar.5  DUST.f	    Makefile	 MODISALBE.f	 OXYG5.f       ROUJBRDF.f     VERSALBE.f
AATSR.f        ENVIRO.f     MAS.f	 MODISBRDF.f	 OXYG6.f       SAND.f	      VERSBRDF.f
ABSTRA.f       EQUIVWL.f    MERIS.f	 MODIS.f	 OZON1.f       SCATRA.f       VERSTOOLS.f
AEROPROF.f     ETM.f	    METEO.f	 MSS.f		 paramdef.inc  SEAWIFS.f      VGT.f
AEROSO.f       Examples     METH1.f	 NIOX1.f	 PLANPOL.f     sixsV1.1       VIIRS.f
AKTOOL.f       GAUSS.f	    METH2.f	 NIOX2.f	 POLDER.f      SOLIRR.f       WALTALBE.

0

# Step 2 - Define functions required in atmospheric correction

**Functions created by Sam Murphy**


*   https://github.com/samsammurphy/gee-atmcorr-S2




In [ ]:
"""
atmospheric.py, Sam Murphy (2016-10-26)
Atmospheric water vapour, ozone and AOT from GEE
Usage
H2O = Atmospheric.water(geom,date)
O3 = Atmospheric.ozone(geom,date)
AOT = Atmospheric.aerosol(geom,date)
"""
class Atmospheric():

  def round_date(date,xhour):
    """
    rounds a date of to the closest 'x' hours
    """
    y = date.get('year')
    m = date.get('month')
    d = date.get('day')
    H = date.get('hour')
    HH = H.divide(xhour).round().multiply(xhour)
    return date.fromYMD(y,m,d).advance(HH,'hour')

  def round_month(date):
    """
    round date to closest month
    """
    # start of THIS month
    m1 = date.fromYMD(date.get('year'),date.get('month'),ee.Number(1))
    # start of NEXT month
    m2 = m1.advance(1,'month')
    # difference from date
    d1 = ee.Number(date.difference(m1,'day')).abs()
    d2 = ee.Number(date.difference(m2,'day')).abs()
    # return closest start of month
    return ee.Date(ee.Algorithms.If(d2.gt(d1),m1,m2))

  def water(geom,date):
    """
    Water vapour column above target at time of image aquisition.
    (Kalnay et al., 1996, The NCEP/NCAR 40-Year Reanalysis Project. Bull.
    Amer. Meteor. Soc., 77, 437-471)

    """
    # Point geometry required
    centroid = geom.centroid()
    # H2O datetime is in 6 hour intervals
    H2O_date = Atmospheric.round_date(date,6)
    # filtered water collection
    water_ic = ee.ImageCollection('NCEP_RE/surface_wv').filterDate(H2O_date, H2O_date.advance(1,'month'))
    # water image
    water_img = ee.Image(water_ic.first())
    # water_vapour at target
    water = water_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('pr_wtr')
    # convert to Py6S units (Google = kg/m^2, Py6S = g/cm^2)
    water_Py6S_units = ee.Number(water).divide(10)
    return water_Py6S_units

  def ozone(geom,date):
    """
    returns ozone measurement from merged TOMS/OMI dataset
    OR
    uses our fill value (which is mean value for that latlon and day-of-year)
    """
    # Point geometry required
    centroid = geom.centroid()

    def ozone_measurement(centroid,O3_date):
      # filtered ozone collection
      ozone_ic = ee.ImageCollection('TOMS/MERGED').filterDate(O3_date, O3_date.advance(1,'month'))
      # ozone image
      ozone_img = ee.Image(ozone_ic.first())
      # ozone value IF TOMS/OMI image exists ELSE use fill value
      ozone = ee.Algorithms.If(ozone_img,\
      ozone_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone'),\
      ozone_fill(centroid,O3_date))
      return ozone

    def ozone_fill(centroid,O3_date):
      """
      Gets our ozone fill value (i.e. mean value for that doy and latlon) you can see it
      1) compared to LEDAPS: https://code.earthengine.google.com/8e62a5a66e4920e701813e43c0ecb83e
      2) as a video: https://www.youtube.com/watch?v=rgqwvMRVguI&feature=youtu.be
      """
      # ozone fills (i.e. one band per doy)
      ozone_fills = ee.ImageCollection('users/samsammurphy/public/ozone_fill').toList(366)
      # day of year index
      jan01 = ee.Date.fromYMD(O3_date.get('year'),1,1)
      doy_index = date.difference(jan01,'day').toInt()# (NB. index is one less than doy, so no need to +1)
      # day of year image
      fill_image = ee.Image(ozone_fills.get(doy_index))
      # return scalar fill value
      return fill_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone')

    # O3 datetime in 24 hour intervals
    O3_date = Atmospheric.round_date(date,24)
    # TOMS temporal gap
    TOMS_gap = ee.DateRange('1994-11-01','1996-08-01')
    # avoid TOMS gap entirely
    ozone = ee.Algorithms.If(TOMS_gap.contains(O3_date),ozone_fill(centroid,O3_date),ozone_measurement(centroid,O3_date))
    # fix other data gaps (e.g. spatial, missing images, etc..)
    ozone = ee.Algorithms.If(ozone,ozone,ozone_fill(centroid,O3_date))
    #convert to Py6S units
    ozone_Py6S_units = ee.Number(ozone).divide(1000)# (i.e. Dobson units are milli-atm-cm )
    return ozone_Py6S_units


  def aerosol(geom,date):
    """
    Aerosol Optical Thickness.
    try:
      MODIS Aerosol Product (monthly)
    except:
      fill value
    """
    def aerosol_fill(date):
      """
      MODIS AOT fill value for this month (i.e. no data gaps)
      """
      return ee.Image('users/samsammurphy/public/AOT_stack')\
               .select([ee.String('AOT_').cat(date.format('M'))])\
               .rename(['AOT_550'])

    def aerosol_this_month(date):
      """
      MODIS AOT original data product for this month (i.e. some data gaps)
      """
      # image for this month
      img =  ee.Image(\
                      ee.ImageCollection('MODIS/006/MOD08_M3')\
                        .filterDate(Atmospheric.round_month(date))\
                        .first()\
                     )
      # fill missing month (?)
      img = ee.Algorithms.If(img,\
                               # all good
                               img\
                               .select(['Aerosol_Optical_Depth_Land_Mean_Mean_550'])\
                               .divide(1000)\
                               .rename(['AOT_550']),\
                              # missing month
                                aerosol_fill(date))
      return img

    def get_AOT(AOT_band,geom):
      """
      AOT scalar value for target
      """
      return ee.Image(AOT_band).reduceRegion(reducer=ee.Reducer.mean(),\
                                 geometry=geom.centroid())\
                                .get('AOT_550')


    after_modis_start = date.difference(ee.Date('2000-03-01'),'month').gt(0)

    AOT_band = ee.Algorithms.If(after_modis_start, aerosol_this_month(date), aerosol_fill(date))

    AOT = get_AOT(AOT_band,geom)

    AOT = ee.Algorithms.If(AOT,AOT,get_AOT(aerosol_fill(date),geom))
    return AOT

Import required libraries

In [ ]:
import ee
from Py6S import *
from datetime import datetime
import math
import pandas as pd
import numpy as np
import os
import sys
import folium
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime, timedelta

**Initialize Google Earth Engine session**

Need enter verification using GEE account

In [ ]:
ee.Authenticate()
ee.Initialize(project='942205441038')

**Py6S function for the atmospheric correction**

In [ ]:
# Define Py6S function
# Modified from https://github.com/ndminhhus/geeguide/blob/master/02.Atm-correction.md

def Py6S(img):
  S2 = ee.Image(img)
  date = S2.date()
  # top of atmosphere reflectance
  toa = S2.divide(10000)

  info = S2.getInfo()['properties']
  scene_date = datetime.utcfromtimestamp(info['system:time_start']/1000)# i.e. Python uses seconds, EE uses milliseconds
  solar_z = info['MEAN_SOLAR_ZENITH_ANGLE']

  h2o = Atmospheric.water(geom,date).getInfo()
  o3 = Atmospheric.ozone(geom,date).getInfo()
  aot = Atmospheric.aerosol(geom,date).getInfo()

  SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
  alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
  km = alt/1000 # i.e. Py6S uses units of kilometers

  # Instantiate
  s = SixS()

  # Atmospheric constituents
  s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
  s.aero_profile = AeroProfile.Maritime # https://github.com/robintw/Py6S/blob/master/Py6S/Params/aeroprofile.py
  s.aot550 = aot

  # Earth-Sun-satellite geometry
  s.geometry = Geometry.User()
  s.geometry.view_z = 0               # always NADIR
  s.geometry.solar_z = solar_z        # solar zenith angle
  s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
  s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
  s.altitudes.set_sensor_satellite_level()
  s.altitudes.set_target_custom_altitude(km)

  def spectralResponseFunction(bandname):
    """
    Extract spectral response function for given band name
    """
    bandSelect = {
        'B1':PredefinedWavelengths.S2A_MSI_01,
        'B2':PredefinedWavelengths.S2A_MSI_02,
        'B3':PredefinedWavelengths.S2A_MSI_03,
        'B4':PredefinedWavelengths.S2A_MSI_04,
        'B5':PredefinedWavelengths.S2A_MSI_05,
        'B6':PredefinedWavelengths.S2A_MSI_06,
        'B7':PredefinedWavelengths.S2A_MSI_07,
        'B8':PredefinedWavelengths.S2A_MSI_08,
        'B8A':PredefinedWavelengths.S2A_MSI_8A,
        'B9':PredefinedWavelengths.S2A_MSI_09,
        'B10':PredefinedWavelengths.S2A_MSI_10,
        'B11':PredefinedWavelengths.S2A_MSI_11,
        'B12':PredefinedWavelengths.S2A_MSI_12,
        }
    return Wavelength(bandSelect[bandname])

  def toa_to_rad(bandname):
    """
    Converts top of atmosphere reflectance to at-sensor radiance
    """
    # solar exoatmospheric spectral irradiance
    ESUN = info['SOLAR_IRRADIANCE_'+bandname]
    solar_angle_correction = math.cos(math.radians(solar_z))
    # Earth-Sun distance (from day of year)
    doy = scene_date.timetuple().tm_yday
    d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
    # conversion factor
    multiplier = ESUN*solar_angle_correction/(math.pi*d**2)
    # at-sensor radiance
    rad = toa.select(bandname).multiply(multiplier)
    return rad

  def surface_reflectance(bandname):
    """
    Calculate surface reflectance from at-sensor radiance given waveband name
    """
    # run 6S for this waveband
    s.wavelength = spectralResponseFunction(bandname)
    s.run()
    # extract 6S outputs
    Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
    Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
    Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
    absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
    scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
    tau2 = absorb*scatter                                #total transmissivity
    # radiance to surface reflectance
    rad = toa_to_rad(bandname)
    ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
    return ref

  # all wavebands
  output = S2.select('QA60')
  for band in ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']:
    # print(band)
    output = output.addBands(surface_reflectance(band))

  return output


# Step 3 - Define functions required to clouds and water mask

Remove cloud and cloud shadow

Modified from:
*   https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
*   https://github.com/ivanhykwong/marine-water-quality-time-series-hk



**Non-water and clouds masks**

In [ ]:
# Funtion to add no-water mask
def add_not_water_mask(img):
    # NDWI
    ndwi = img.normalizedDifference(['B3', 'B8']).rename('NDWI')
    # mNDWI
    mndwi = img.normalizedDifference(['B3', 'B11']).rename('mNDWI')

    mndwi_water = mndwi.gt(-0.05)
    ndwi_water = ndwi.gt(0.3)

    combined_water = mndwi_water.Or(ndwi_water).rename('COMBINED_WATER')
    not_water = combined_water.Not().rename('NOT_WATER')
    return img.addBands(not_water)

# Funtion to add cloud mask
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    # Condition s2cloudless by the probability threshold value (1 clouds)
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

# Funtion to add cloud shadow mask
def add_shadow_bands(img):
    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 10})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

# Funtion to add combined mask
def add_cld_shdw_mask(img):

    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)
    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)
    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/10)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 10})
        .rename('cloudmask'))

    # Add final mask
    return img_cloud_shadow.addBands(is_cld_shdw)

**Import colecction and apply corrections**

In [ ]:
# Function to apply corrections and add mask
def apply_corrections(image):
    # Apply Py6S corrections
    corrected_img = Py6S(image)
    print("Bands after Py6S:", corrected_img.bandNames().getInfo())
    # Add masks
    masked_img = add_not_water_mask(corrected_img)
    print("Bands after add_not_water_mask:", masked_img.bandNames().getInfo())
    final_img = add_cld_shdw_mask(masked_img)
    print("Bands after add_cld_shdw_mask:", final_img.bandNames().getInfo())
    return final_img

In [ ]:
# Import and filter S2 and s2cloudless
def get_s2_sr_cld_col(aoi, dates, search_window=6):
    # Process each date in the provided list to find and correct images within the search window
    def process_date(date_str):
      # Calculate the search window around each target date
        target_date = datetime.strptime(date_str, '%Y-%m-%d')
        start_search = target_date - timedelta(days=search_window)
        end_search = target_date + timedelta(days=search_window)

      # Filter Sentinel-2 surface reflectance images by AOI and date, with cloud cover threshold
        s2_sr_col = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')\
                     .filterBounds(aoi)\
                     .filterDate(start_search.strftime('%Y-%m-%d'), end_search.strftime('%Y-%m-%d'))\
                     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))

        # Filter s2cloudless images for the same AOI and date range
        s2_cloudless_col = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')\
                            .filterBounds(aoi)\
                            .filterDate(start_search.strftime('%Y-%m-%d'), end_search.strftime('%Y-%m-%d'))

        # Join s2 surface reflectance and s2cloudless collections on image index
        joined_col = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
            'primary': s2_sr_col,
            'secondary': s2_cloudless_col,
            'condition': ee.Filter.equals(**{
                'leftField': 'system:index',
                'rightField': 'system:index'
            })
        }))

        # Map over the collection to calculate the absolute difference in time from the target date
        def date_diff(image):
            image_date = ee.Date(image.get('system:time_start'))
            diff = ee.Number(target_date.timestamp() * 1000).subtract(image_date.millis()).abs()
            return image.set('dateDiff', diff)
        # Sort by time difference and select the first (closest) image
        nearest_image = joined_col.map(date_diff).sort('dateDiff').first()
        # Print selected date and cloudiness for verification
        selected_date = ee.Date(nearest_image.get('system:time_start')).format('YYYY-MM-dd')
        cloud_percentage = nearest_image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
        print(f'Original date: {date_str}, Select date: {selected_date.getInfo()} Cloudiness: {cloud_percentage}%')

        # Apply corrections and masks to the nearest image.
        return apply_corrections(nearest_image)
    return [process_date(date) for date in dates]

# Step 4 - Apply functions to the area and date selected

**Filter by date and aoi**

In [ ]:
# Define the area of interest (AOI)
Clip = ee.FeatureCollection('projects/epvillanueva/assets/Clip')
AOI = Clip.geometry()
geom = AOI
# Dates of in-situ information to train and test the future machine learning model
DATES = ['2022-11-24', '2023-01-27', '2023-02-05', '2023-03-17', '2023-03-27', '2023-04-11', '2023-04-26', '2023-05-16', '2023-06-05', '2023-07-05']
CLD_PRB_THRESH = 40  #	Cloud probability (%); values greater than are considered cloud
NIR_DRK_THRESH = 0.5 # Near-infrared reflectance; values less than are considered potential cloud shadow
CLD_PRJ_DIST = 2 # Maximum distance (km) to search for cloud shadows from cloud edges
BUFFER = 15 # Distance (m) to dilate the edge of cloud-identified objects
CLOUD_FILTER = 50

**Apply corrections and add mask to visualization**

In [58]:
# Apply functions to collections
collections = get_s2_sr_cld_col(AOI, DATES)

Original date: 2022-11-24, Select date: 2022-11-22 Cloudiness: 38.6851807335749%
Bands after Py6S: ['QA60', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
Bands after add_not_water_mask: ['QA60', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'NOT_WATER']
Bands after add_cld_shdw_mask: ['QA60', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'NOT_WATER', 'probability', 'clouds', 'dark_pixels', 'cloud_transform', 'shadows', 'cloudmask']
Original date: 2023-01-27, Select date: 2023-01-26 Cloudiness: 0.307621597213263%
Bands after Py6S: ['QA60', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
Bands after add_not_water_mask: ['QA60', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'NOT_WATER']
Bands after add_cld_shdw_mask: ['QA60', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'NOT_WA

**Visualization**

In [ ]:
# Function to add Google Earth Engine image layers to Folium maps
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):

    # Generate map ID and token for the Earth Engine image
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    # Add the Earth Engine tile layer to the Folium map
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
    ).add_to(self)

# Extend the Folium Map class to include the add_ee_layer method
folium.Map.add_ee_layer = add_ee_layer

# Function to generate and add specific layers of interest to the map for a given image
def generate_layers_for_date(img, map_object):

    # Self-mask layers to visualize specific features
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability').selfMask()
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform').selfMask()
    not_water = img.select('NOT_WATER').selfMask()

    # Add specific layers to the map with visualization parameters
    map_object.add_ee_layer(img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.25, 'gamma': 1.1}, 'S2 image')
    map_object.add_ee_layer(cloudmask, {'palette': 'orange'}, 'Cloud Mask')
    map_object.add_ee_layer(not_water, {'palette': 'blue'}, 'Not Water')
    # Uncomment to add additional layers
    # map_object.add_ee_layer(probability, {'min': 0, 'max': 100}, 'Probability (cloud)')
    # map_object.add_ee_layer(clouds, {'palette': 'e056fd'}, 'Clouds')
    # map_object.add_ee_layer(cloud_transform, {'min': 0, 'max': 1, 'palette': ['white', 'black']}, 'Cloud Transform')
    # map_object.add_ee_layer(dark_pixels, {'palette': 'orange'}, 'Dark Pixels')
    # map_object.add_ee_layer(shadows, {'palette': 'yellow'}, 'Shadows')

# Function to visualize layers with a date selector for user interaction
def display_cloud_layers_with_date_selector(collections, dates, aoi):
    def update_map(date_index):
       # Create a new map object for the selected date
        center = aoi.centroid(10).coordinates().reverse().getInfo()
        m = folium.Map(location=center, zoom_start=14)

        # Generate and add layers to the map based on the selected date
        img = collections[date_index]
        generate_layers_for_date(img, m)

        # Add layer control to toggle layers on/off
        m.add_child(folium.LayerControl())

        # Display the updated map
        display(m)

    # Create a dropdown widget for date selection
    date_selector = widgets.Dropdown(
        options=[(date, i) for i, date in enumerate(dates)],
        value=0,
        description='Select Date:',
    )

    # Link the date selector widget to the map update function
    widgets.interact(update_map, date_index=date_selector)

In [ ]:
display_cloud_layers_with_date_selector(collections, DATES, AOI)

interactive(children=(Dropdown(description='Select Date:', options=(('2022-11-24', 0), ('2023-01-27', 1), ('20…

**Apply the masks and clip after confirming correct use with the display**

In [ ]:
# Function to apply masks and clip
def apply_masks_and_clip(img):
    # Apply non-water, cloud, and shadow masks to the image.
    masked_img = add_cld_shdw_mask(add_not_water_mask(img))
    # print("Bands after masks:", masked_img.bandNames().getInfo())

    # Invert masks to keep only unmasked areas (0 = no clouds, no shadows, and water presence)
    cloud_shadow_mask = masked_img.select('cloudmask').Not()
    water_mask = masked_img.select('NOT_WATER').Not()

    # Combine masks to retain only unmasked areas (no clouds, no shadows, and including water)
    final_mask = cloud_shadow_mask.And(water_mask)
    # Update the image mask with the final mask and clip to the area of interest (AOI)
    final_img = masked_img.updateMask(final_mask).clip(AOI)

    # Select bands
    band_names = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
    selected_final_img = final_img.select(band_names)
    print("Bands of processed images :", selected_final_img.bandNames().getInfo())

    return selected_final_img

# Function to visualize the processed images
def visualize_processed_images(processed_collections, dates, aoi):
    def update_map(date_index):
        # Create a new map object for the selected date
        center = aoi.centroid(10).coordinates().reverse().getInfo()
        m = folium.Map(location=center, zoom_start=14)

        # Add only the processed and clipped image to the map
        img = processed_collections[date_index]
        m.add_ee_layer(img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.25, 'gamma': 1.1}, 'Processed S2 image')
        # Add layer control for toggling layers on and off
        m.add_child(folium.LayerControl())

        # Display the updated map
        display(m)

    # Create a dropdown widget for selecting dates
    date_selector = widgets.Dropdown(
        options=[(date, i) for i, date in enumerate(dates)],
        value=0,
        description='Select Date:',
    )

    # Link the date selector widget to the map updating function
    widgets.interact(update_map, date_index=date_selector)

In [59]:
# Process images with masks and clipping
processed_collections = [apply_masks_and_clip(img) for img in collections]

# Visualize the processed images
visualize_processed_images(processed_collections, DATES, AOI)

Bands of processed images : ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
Bands of processed images : ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
Bands of processed images : ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
Bands of processed images : ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
Bands of processed images : ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
Bands of processed images : ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
Bands of processed images : ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
Bands of processed images : ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
Bands of processed images : ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
B

interactive(children=(Dropdown(description='Select Date:', options=(('2022-11-24', 0), ('2023-01-27', 1), ('20…

# Step 5 - Extract satellite data at in-situ locations

After processing the date images of the in-situ data, the information of each pixel that matches in date and coordinate with the in-situ data is extracted to train and evaluate machine learning models to predict the variables of interest.

In [ ]:
# Define the FeatureCollection of stations from a specified Earth Engine asset.
stations = ee.FeatureCollection('projects/epvillanueva10/assets/Stations')

# Function to extract data from images at station locations.
def extract_data_for_stations(processed_collections, stations):
    all_results = []

    # Retrieve a list of station IDs.
    station_ids = stations.aggregate_array('system:index').getInfo()

    for img in processed_collections:
        # Extract the image date.
        image_date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()

        for station_id in station_ids:
            # Get the specific station by ID.
            station = stations.filter(ee.Filter.eq('system:index', station_id)).first()
            # Retrieve station coordinates and name.
            coords = station.geometry().coordinates().getInfo()
            station_name = station.get('Estacion').getInfo()

            # Extract image values at the station coordinates.
            values = img.reduceRegion(
                ee.Reducer.toList(),
                ee.Geometry.Point(coords),
                10
            ).getInfo()

            # Prepare the results dictionary, converting empty lists to NaN.
            result = {
                'Station': station_name,
                'Date': image_date,
                'coordinates': coords
            }
            for band in values.keys():
                result[band] = values[band][0] if values[band] else float('nan')

            all_results.append(result)

    return all_results

In [ ]:
# Apply the function to the entire collection and retrieve station data.
station_data = extract_data_for_stations(processed_collections, stations)

# Step 6 - Prepare satellite and in-situ data for the machine learning models

**Load satellite and in-situ data**

In [ ]:
import pandas as pd
# Load satellite data from CSV file in Google Drive
satellite_data = pd.read_csv('/content/drive/MyDrive/Mallorquin/data/satellite_data.csv')
satellite_data['Date'] = pd.to_datetime(satellite_data['Date'])

# Load in-situ data from CSV file in Google Drive
in_situ_data = pd.read_csv('/content/drive/MyDrive/Mallorquin/data/insitu_data.csv')
in_situ_data['Date'] = pd.to_datetime(in_situ_data['Date'])

# Display information about the in-situ data DataFrame
in_situ_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Station      300 non-null    int64         
 1   Latitude     300 non-null    float64       
 2   Longitude    300 non-null    float64       
 3   Date         300 non-null    datetime64[ns]
 4   Temperature  252 non-null    float64       
 5   Salinity     240 non-null    float64       
 6   CSS          217 non-null    float64       
 7   DO           192 non-null    float64       
 8   Chl_a        12 non-null     float64       
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 21.2 KB


**Adjust the dates to combine the in situ data with the satellite data**

In [ ]:
# Unique dates in satellite data
unique_satellite_dates = satellite_data['Date'].unique()
print(unique_satellite_dates)

['2022-11-22T00:00:00.000000000' '2023-01-26T00:00:00.000000000'
 '2023-02-05T00:00:00.000000000' '2023-03-17T00:00:00.000000000'
 '2023-03-27T00:00:00.000000000' '2023-04-11T00:00:00.000000000'
 '2023-04-26T00:00:00.000000000' '2023-05-11T00:00:00.000000000'
 '2023-06-10T00:00:00.000000000' '2023-07-05T00:00:00.000000000']


In [ ]:
# Unique dates in in-situ data
unique_in_situ_dates = in_situ_data['Date'].unique()
print(unique_in_situ_dates)

['2022-11-22T00:00:00.000000000' '2023-01-26T00:00:00.000000000'
 '2023-02-05T00:00:00.000000000' '2023-03-17T00:00:00.000000000'
 '2023-03-27T00:00:00.000000000' '2023-04-11T00:00:00.000000000'
 '2023-04-26T00:00:00.000000000' '2023-05-16T00:00:00.000000000'
 '2023-06-05T00:00:00.000000000' '2023-07-05T00:00:00.000000000']


In [ ]:
# Copy the in situ data to adjust the dates to combine the in situ with the satellite data for the closest dates
adjusted_in_situ = in_situ_data.copy()
adjusted_in_situ.loc[adjusted_in_situ['Date'] == '2022-11-24', 'Date'] = pd.to_datetime('2022-11-22')
adjusted_in_situ.loc[adjusted_in_situ['Date'] == '2023-01-27', 'Date'] = pd.to_datetime('2023-01-26')
adjusted_in_situ.loc[adjusted_in_situ['Date'] == '2023-02-03', 'Date'] = pd.to_datetime('2023-02-05')
adjusted_in_situ.loc[adjusted_in_situ['Date'] == '2023-05-16', 'Date'] = pd.to_datetime('2023-05-11')
adjusted_in_situ.loc[adjusted_in_situ['Date'] == '2023-06-05', 'Date'] = pd.to_datetime('2023-06-10')

# Merge in-situ data with satellite data based on 'Station' and 'Date' columns
combined_data = pd.merge(adjusted_in_situ, satellite_data, on=['Station', 'Date'], how='inner')
# Drop columns that are duplicated in both or don't have surface information
combined_data.drop(['coordinates', 'Station', 'Latitude', 'Longitude', 'Date', 'B10', 'DO'], axis=1, inplace=True)

# Adjust the scale of reflectance values by multiplying each band by 10000
# This converts the reflectance back to the original scale used before atmospheric correction
bands = ['B' + str(b) for b in [1, 2, 3, 4, 5, 6, 7, 8, '8A', 9, 11, 12]]
combined_data[bands] = (combined_data[bands]*10000)

# Display the first few rows of the combined data
combined_data.head()

,Temperature,Salinity,CSS,Chl_a,B1,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,B9
0,30.721048,28.544563,0.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29.548711,29.775934,0.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29.436773,29.791698,1.14,NaN,916.052976,491.008186,458.16432,961.593349,980.985059,816.02325,782.316097,724.203465,771.352872,715.700342,664.457103,2858.698686
3,29.168811,28.516698,0.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29.063744,29.880780,1.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The observed NaN values represent the absence of in-situ data for some dates and/or stations, and the absence of satellite data due to the masks applied.


**Save .csv files with the data of each variable in Google Drive for machine learning models**

In [ ]:
# Drop specified columns from the Salinity
salinity = combined_data.drop(['CSS', 'Temperature', 'Chl_a'], axis=1)
# Remove rows with missing values
salinity.dropna(inplace=True)
# Save the DataFrame to CSV
salinity.to_csv('/content/drive/My Drive/Mallorquin/data/Salinity.csv', index=False)

# Drop specified columns from the Temperature
temperature = combined_data.drop(['Salinity', 'CSS', 'Chl_a'], axis=1)
# Remove rows with missing values
temperature.dropna(inplace=True)
# Save the DataFrame to CSV
temperature.to_csv('/content/drive/My Drive/Mallorquin/data/Temperature.csv', index=False)

# Drop specified columns from the CSS
css = combined_data.drop(['Salinity', 'Temperature', 'Chl_a'], axis=1)
# Remove rows with missing values
css.dropna(inplace=True)
# Save the DataFrame to CSV
css.to_csv('/content/drive/My Drive/Mallorquin/data/CSS.csv', index=False)

# Drop specified columns from the Chl_a
chl = combined_data.drop(['Salinity', 'CSS', 'Temperature'], axis=1)
# Remove rows with missing values
chl.dropna(inplace=True)
# Save the DataFrame to CSV
chl.to_csv('/content/drive/My Drive/Mallorquin/data/Chl_a.csv', index=False)

# Step 7 - Apply correction and mask functions to entire collection for export

The correction and masking functions already established will be used on a selection of images from the collection. One image will be chosen for each month, the one with the lowest percentage of cloudiness.

To obtain these images, the function "get_s2_sr_cld_col" will be updated to the function "get_least_cloudy_image_per_month".

In [ ]:
# Function to retrieve the least cloudy image for each month within a specified date range for a given area of interest (AOI).
def get_least_cloudy_image_per_month(start_date, end_date, aoi):
    # Generate a range of dates from start to end with one-month intervals.
    date_range = pd.date_range(start_date, end_date, freq='MS')
    selected_images = []  # To store selected images.
    image_dates = []  # To store the dates of selected images.

    # Iterate through each month in the date range.
    for date in date_range:
        # Define the start and end of the month.
        start_month = date.strftime('%Y-%m-%d')
        end_month = (date + pd.offsets.MonthEnd()).strftime('%Y-%m-%d')

        # Filter Sentinel-2 surface reflectance images by AOI and date, sorting by cloud coverage.
        s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                      .filterBounds(aoi)
                      .filterDate(start_month, end_month)
                      .sort('CLOUDY_PIXEL_PERCENTAGE'))

        # Filter corresponding cloud probability images for the same AOI and date range.
        s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
                             .filterBounds(aoi)
                             .filterDate(start_month, end_month))

        # Join the surface reflectance and cloud probability collections using image indices.
        joined_col = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(
            primary=s2_sr_col,
            secondary=s2_cloudless_col,
            condition=ee.Filter.equals(
                leftField='system:index',
                rightField='system:index'
            )))

        # Check if there are any images for the month.
        count = joined_col.size().getInfo()
        if count > 0:
            least_cloudy_image = joined_col.first()

            # Extract and print date and cloudiness information.
            date_info = least_cloudy_image.date().format('YYYY-MM-dd').getInfo()
            cloud_info = least_cloudy_image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
            print(f'Date: {date_info}, Cloudiness Percentage: {cloud_info}%')


            # Apply corrections (e.g., atmospheric corrections, masks) to the image and add to the list.
            corrected_image = Py6S(least_cloudy_image)
            final_image = apply_masks_and_clip(corrected_image)

            selected_images.append(final_image)
            image_dates.append(date_info)
        else:
            print(f"No images available for the month: {start_month}")

    return selected_images, image_dates

In [52]:
# Apply functions to collections
collections_per_month, image_dates = get_least_cloudy_image_per_month('2016-01-01', '2023-12-31', AOI)

Date: 2016-01-23, Cloudiness Percentage: 0.4917%
Bands after Py6S: ['QA60', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
Bands after add_not_water_mask: ['QA60', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'NOT_WATER']
Bands after add_cld_shdw_mask: ['QA60', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'NOT_WATER', 'probability', 'clouds', 'dark_pixels', 'cloud_transform', 'shadows', 'cloudmask']
Bands of processed images : ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
No images available for the month: 2016-02-01
Date: 2016-03-23, Cloudiness Percentage: 0%
Bands after Py6S: ['QA60', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
Bands after add_not_water_mask: ['QA60', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'NOT_WATER']
Bands after add_cld_shdw_mask: ['QA60', 'B

In [57]:
# Function to export the collection to a GEE asset collection
def export_to_ee_collection(image_collection, image_dates, aoi):

    for img, date in zip(image_collection, image_dates):
        asset_id = f"{'projects/epvillanueva10/assets/Mallorquin'}/{date}"

        # Define the export parameters
        export_task = ee.batch.Export.image.toAsset(
            image=img.toFloat(),
            description=date,
            assetId=asset_id,
            region=aoi,
            scale=20,
            maxPixels=1e13
        )

        # Start the export task
        export_task.start()
        print(f'Exporting image {date} to {asset_id}')

export_to_ee_collection(collections_per_month, image_dates, AOI)

Exporting image 2016-01-23 to projects/epvillanueva10/assets/Mallorquin/2016-01-23
Exporting image 2016-03-23 to projects/epvillanueva10/assets/Mallorquin/2016-03-23
Exporting image 2016-04-22 to projects/epvillanueva10/assets/Mallorquin/2016-04-22
Exporting image 2016-05-22 to projects/epvillanueva10/assets/Mallorquin/2016-05-22
Exporting image 2016-06-11 to projects/epvillanueva10/assets/Mallorquin/2016-06-11
Exporting image 2016-07-21 to projects/epvillanueva10/assets/Mallorquin/2016-07-21
Exporting image 2016-08-30 to projects/epvillanueva10/assets/Mallorquin/2016-08-30
Exporting image 2016-09-19 to projects/epvillanueva10/assets/Mallorquin/2016-09-19
Exporting image 2016-10-29 to projects/epvillanueva10/assets/Mallorquin/2016-10-29
Exporting image 2016-11-08 to projects/epvillanueva10/assets/Mallorquin/2016-11-08
Exporting image 2016-12-18 to projects/epvillanueva10/assets/Mallorquin/2016-12-18
Exporting image 2017-01-27 to projects/epvillanueva10/assets/Mallorquin/2017-01-27
Expo